### Shared Word File Process

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Spanish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # always must be False in this part
native_word = True # True for adding native word
etymology_word = True  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.
print(f"{file_ext}")

1


In [4]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)

### Twogram In Threegram

In [5]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,que,37853284
1,de,37809537
2,no,33043466
3,a,25439588
4,la,24024343
...,...,...
446459,bibe,5
446460,dejugo,5
446461,bibbi,5
446462,bibberman,5


In [6]:
df_shared_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency{file_ext}.xlsx")
#df_shared_file = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx")
df_shared_file

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,que,37853284.0,lo que,1485227.0,por qué no,344733,eso es lo que,101130.0,por qué no me lo,11245.0,está bien,506251.0
1,de,37809537.0,por qué,1460656.0,es lo que,296245,qué es lo que,60509.0,eso no es lo que,8546.0,de acuerdo,344687.0
2,no,33043466.0,no lo,1252184.0,eso es lo,113501,por qué no me,39905.0,eso es lo que me,6031.0,por qué,330492.0
3,a,25439588.0,no es,1114995.0,qué es eso,109973,no es lo que,28512.0,qué es lo que te,4002.0,qué es eso,88217.0
4,la,24024343.0,está bien,963477.0,eso no es,92222,por qué no lo,22222.0,y eso es lo que,3529.0,eso es,87824.0
...,...,...,...,...,...,...,...,...,...,...,...,...
121577,NaN,NaN,NaN,NaN,es su chip,3,NaN,NaN,NaN,NaN,NaN,NaN
121578,NaN,NaN,NaN,NaN,solo con terror,3,NaN,NaN,NaN,NaN,NaN,NaN
121579,NaN,NaN,NaN,NaN,de dinosaurio de,3,NaN,NaN,NaN,NaN,NaN,NaN
121580,NaN,NaN,NaN,NaN,de dinosaurio y,3,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [8]:
def word_in_wordgroup(df, list_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            #word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(10)  # Option
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(100) 
        except:
            pass        
        word_in_word_cluster.insert(0,f"{list_column}",i)
        df_word_result = pd.concat([df_word_result,word_in_word_cluster], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [9]:
#df_shared_count = word_count_result(df_shared_file,["threegram"])
#df_shared_count

In [10]:
df_shared_file["twogram"].nunique()

44988

In [11]:
df_two_in_three = word_in_wordgroup(df_shared_file, "twogram", "threegram")
df_two_in_three

,twogram,threegram
0,lo que,es lo que
1,lo que,de lo que
2,lo que,lo que me
3,lo que,lo que te
4,lo que,lo que es
...,...,...
203393,pragmático a,pragmático a su
203394,control maníaco,su control maníaco
203395,ritmo bien,el ritmo bien
203396,magnetismo por,magnetismo por qué


In [12]:
df_two_in_three["twogram"].nunique()

36648

In [13]:
df_shared_select_twogram = df_shared_file.loc[:,["twogram","freq_twogram"]]
df_shared_select_twogram

,twogram,freq_twogram
0,lo que,1485227.0
1,por qué,1460656.0
2,no lo,1252184.0
3,no es,1114995.0
4,está bien,963477.0
...,...,...
121577,NaN,NaN
121578,NaN,NaN
121579,NaN,NaN
121580,NaN,NaN


In [14]:
set_shared_twogram = set(df_shared_select_twogram["twogram"])
set_two_three = set(df_two_in_three["twogram"])

In [15]:
df_twogram_in_threegram = pd.DataFrame(set_two_three, columns=["twogram"])  # columns=["twogram_in_threegram"]
df_twogram_in_threegram

,twogram
0,de concepto
1,capitalista que
2,resistencia física
3,el potencial
4,de doctorado
...,...
36643,una actriz
36644,buffet no
36645,clave personal
36646,y navegación


In [16]:
df_twogram_in_threegram_freq = pd.merge(df_twogram_in_threegram, df_shared_select_twogram, how="left", on="twogram")
df_twogram_in_threegram_freq.sort_values(by="freq_twogram", ascending=False, inplace=True)
df_twogram_in_threegram_freq.rename(columns={"twogram":"twogram_in_threegram","freq_twogram":"freq_two_in_three"}, inplace=True)
df_twogram_in_threegram_freq.reset_index(drop=True, inplace=True)
df_twogram_in_threegram_freq

,twogram_in_threegram,freq_two_in_three
0,lo que,1485227.0
1,por qué,1460656.0
2,no lo,1252184.0
3,no es,1114995.0
4,está bien,963477.0
...,...,...
36643,acuerdo súper,3.0
36644,qué grasa,3.0
36645,en chasis,3.0
36646,su burgués,3.0


In [17]:
df_twogram_diff = pd.DataFrame(set_shared_twogram.difference(set_two_three), columns=["twogram"])
df_twogram_diff

,twogram
0,NaN
1,equipo comandante
2,imperio sí
3,no hamburguesa
4,psicólogo eso
...,...
8336,los compartimiento
8337,en radioactivo
8338,a radar
8339,sí masajista


In [18]:
df_twogram_diff_freq = pd.merge(df_twogram_diff, df_shared_select_twogram, how="left", on="twogram")
df_twogram_diff_freq.sort_values(by="freq_twogram", ascending=False, inplace=True)
df_twogram_diff_freq.reset_index(drop=True, inplace=True)
df_twogram_diff_freq

,twogram,freq_twogram
0,hola princesa,381.0
1,hola maestro,233.0
2,hola red,215.0
3,hola norma,188.0
4,embargo los,174.0
...,...,...
84929,NaN,NaN
84930,NaN,NaN
84931,NaN,NaN
84932,NaN,NaN


In [19]:
df_shared_file["twogram"] = df_twogram_diff_freq["twogram"]
df_shared_file["freq_twogram"] = df_twogram_diff_freq["freq_twogram"]

In [20]:
df_shared_twogram_process = pd.concat([df_shared_file,df_twogram_in_threegram_freq], axis=1)
df_shared_twogram_process

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence,twogram_in_threegram,freq_two_in_three
0,que,37853284.0,hola princesa,381.0,por qué no,344733,eso es lo que,101130.0,por qué no me lo,11245.0,está bien,506251.0,lo que,1485227.0
1,de,37809537.0,hola maestro,233.0,es lo que,296245,qué es lo que,60509.0,eso no es lo que,8546.0,de acuerdo,344687.0,por qué,1460656.0
2,no,33043466.0,hola red,215.0,eso es lo,113501,por qué no me,39905.0,eso es lo que me,6031.0,por qué,330492.0,no lo,1252184.0
3,a,25439588.0,hola norma,188.0,qué es eso,109973,no es lo que,28512.0,qué es lo que te,4002.0,qué es eso,88217.0,no es,1114995.0
4,la,24024343.0,embargo los,174.0,eso no es,92222,por qué no lo,22222.0,y eso es lo que,3529.0,eso es,87824.0,está bien,963477.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121577,NaN,NaN,NaN,NaN,es su chip,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121578,NaN,NaN,NaN,NaN,solo con terror,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121579,NaN,NaN,NaN,NaN,de dinosaurio de,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121580,NaN,NaN,NaN,NaN,de dinosaurio y,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df_shared_twogram_process.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency{file_ext}2.xlsx", index=False)

##### Concat Result With Comma

In [22]:
df_word_order_twogram = word_in_wordgroup(df_shared_twogram_process, "word", "twogram")
df_word_order_threegram = word_in_wordgroup(df_shared_twogram_process, "word", "threegram") 
df_word_order_fourgram = word_in_wordgroup(df_shared_twogram_process, "word", "fourgram") 
df_word_order_fivegram = word_in_wordgroup(df_shared_twogram_process, "word", "fivegram")
df_word_order_sentence = word_in_wordgroup(df_shared_twogram_process, "word", "sentence")
df_word_order_twogram_threegram = word_in_wordgroup(df_shared_twogram_process, "word", "twogram_in_threegram") 

In [23]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["word"])["sentence"].apply(", ".join).reset_index()
df_word_order_join_twogram_threegram = df_word_order_twogram_threegram.groupby(["word"])["twogram_in_threegram"].apply(", ".join).reset_index()

In [24]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence,df_word_order_join_twogram_threegram]

In [25]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all  

,word,twogram,threegram,fourgram,fivegram,sentence,twogram_in_threegram
0,a,"a bebé, a futbol, a transbordador, a playa, a ...","a la policía, a qué te, a punto de, a lo que, ...","a lo que me, está a punto de, es a lo que, a l...","es a lo que me, eso es a lo que, no es a lo qu...","a qué, a la carga, a mi, a que sí, a la policí...","a la, a mi, a los, a un, a su, a qué, a una, y..."
1,abogado,"hola abogado, adiós abogado, bravo abogado, si...","a un abogado, a mi abogado, el abogado de, con...","el abogado de la, es el abogado de, con el abo...","el abogado de la familia, abogado con una cris...","un abogado, es abogado, el abogado, mi abogado...","un abogado, mi abogado, el abogado, su abogado..."
2,absorber,"bien absorber, absorber una, absorber radiació...","que absorber los, de absorber energía, para ab...","es para absorber la, mi cuerpo a absorber, cue...","a mi cuerpo a absorber, a absorber el suero y,...",bien absorber,"para absorber, absorber los, que absorber, abs..."
3,absurdo,"absurdo cuerpo, objeto absurdo, absurdo arbitr...","eso es absurdo, no es absurdo, es absurdo no, ...","pero eso es absurdo, no eso es absurdo, eso es...","lo absurdo de lo que, absurdo en la forma en, ...","es absurdo, eso es absurdo, qué absurdo, pero ...","es absurdo, qué absurdo, absurdo que, absurdo ..."
4,acacia,"acacia en, de acacia, acacia lo","la acacia que, jazmín y acacia, en una acacia,...",de jazmín y acacia,el de jazmín y acacia,el de jazmín y acacia,"la acacia, acacia que, y acacia, una acacia"
...,...,...,...,...,...,...,...
1474,totalitario,NaN,"un dictador totalitario, un estado totalitario","de un dictador totalitario, en un estado total...",NaN,NaN,"dictador totalitario, estado totalitario"
1475,turbina,NaN,"una turbina de, de la turbina, es una turbina,...","es una turbina de, la turbina de un, por la tu...","por la turbina de un, lo que es la turbina, en...","la turbina, una turbina, lo que es la turbina,...","la turbina, una turbina, turbina de, de turbin..."
1476,variante,NaN,"una variante de, es una variante, es la varian...","es una variante de, una variante de la, varian...","un virus de la variante, una variante de qué e...","y la variante, una variante","una variante, variante de, la variante, varian..."
1477,voltio,NaN,"y el voltio, con un voltio, un voltio por",con un voltio por,NaN,un voltio,"un voltio, el voltio, voltio por"


In [26]:
# Option For Word Frequency
if shared_word_frequency:
    df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="inner", on="word")
    df_word_order_join_all.drop_duplicates(inplace=True)
    df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram","fourgram","fivegram","sentence","twogram_in_threegram"]]
    df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
    df_word_order_join_all.reset_index(drop=True, inplace=True)
else:
    pass

df_word_order_join_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence,twogram_in_threegram
0,que,37853284,"que familia, que justa, que información, que c...","es lo que, de lo que, lo que me, lo que te, qu...","eso es lo que, qué es lo que, no es lo que, es...","eso no es lo que, eso es lo que me, qué es lo ...","por que, es que, que es eso, que es, que bien,...","lo que, que no, que te, que me, que lo, que es..."
1,de,37809537,"de diálogo, de cubo, de lesión, de magnetismo,...","de lo que, a punto de, de qué se, de que no, n...","de acuerdo de acuerdo, está a punto de, una ta...","eso es de lo que, de eso es de lo, por el bien...","de acuerdo, de qué, sí de acuerdo, de acuerdo ...","de la, de acuerdo, de qué, de los, de que, de ..."
2,no,33043466,"no condesa, no nuclear, no director, declarado...","por qué no, eso no es, no no lo, no es un, no ...","por qué no me, no es lo que, por qué no lo, po...","por qué no me lo, eso no es lo que, no es eso ...","por qué no, no está bien, no lo se, no lo es, ...","no lo, no es, no me, que no, no te, no se, qué..."
3,a,25439588,"a bebé, a futbol, a transbordador, a playa, a ...","a la policía, a qué te, a punto de, a lo que, ...","a lo que me, está a punto de, es a lo que, a l...","es a lo que me, eso es a lo que, no es a lo qu...","a qué, a la carga, a mi, a que sí, a la policí...","a la, a mi, a los, a un, a su, a qué, a una, y..."
4,la,24024343,"la destino, almirante la, la ilegal, marcos la...","a la policía, está en la, no es la, a la escue...","la forma en que, por qué no la, no está en la,...","no está en la lista, es la forma en que, por e...","la policía, a la carga, es la policía, la qué,...","a la, de la, en la, es la, por la, que la, con..."
...,...,...,...,...,...,...,...,...
1474,briqueta,18,NaN,es una briqueta,club es una briqueta,NaN,NaN,una briqueta
1475,caroteno,14,NaN,"beta caroteno y, de beta caroteno",de beta caroteno y,NaN,NaN,"beta caroteno, caroteno y"
1476,rosetón,9,el rosetón,NaN,NaN,NaN,NaN,NaN
1477,megavatio,5,un megavatio,NaN,NaN,NaN,un megavatio,NaN


In [27]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency{file_ext}2.xlsx", index=False)

#### Copy Move And Delete

In [28]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*{file_ext}2.xlsx")
output_file

['Spanish_Turkish_Shared_Result_With_Frequency12.xlsx',
 'Spanish_Turkish_Shared_Join_Result_Without_Frequency12.xlsx']

In [29]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [30]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

### Prefix Suffix Result Analysis

In [105]:
import os
import multiprocessing
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil

In [106]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "Italian"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# word sample
word_sample = True  # True, False
word_sample_num = 20

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = True  # True, False  # always must be True in this part
native_word = False # True for adding native word
etymology_word = True  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.
print(f"{file_ext}")

4


In [107]:
def word_in_wordgroup(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            if word_sample:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)  # Option
            else:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_word_result = pd.concat([df_word_result,word_in_word_cluster], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [108]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988212,karneleme,5
988213,karnaya,5
988214,dörtlümüzün,5
988215,karnavalınız,5


In [109]:
df_word_prefix_suffix = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
df_word_prefix_suffix = df_word_prefix_suffix[["search_word","word"]]
df_word_prefix_suffix

,search_word,word
0,ama,ama
1,bana,bana
2,ben,ben
3,ben,bence
4,ben,bende
...,...,...
162,şey,şeyler
163,şey,şeylerden
164,şey,şeylere
165,şey,şeyleri


In [110]:
df_ety_prefix_suffix = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
df_ety_prefix_suffix = df_ety_prefix_suffix[["search_word","word"]]
df_ety_prefix_suffix

,search_word,word
0,abone,abone
1,abone,abonelik
2,abone,aboneliği
3,absorbe,absorbe
4,absürt,absürt
...,...,...
4835,şövalye,şövalyenin
4836,şövalye,şövalyesi
4837,şırınga,şırınga
4838,şırınga,şırıngayla


In [111]:
if file_ext == "3":
    df_all_word = pd.concat([df_word_prefix_suffix,df_ety_prefix_suffix],axis=0)
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
elif file_ext == "4":
    df_all_word = df_ety_prefix_suffix
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
elif file_ext == "5":
    df_all_word = df_word_prefix_suffix
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
else:
    pass
    
df_all_word

,search_word,word
0,abone,abone
1,abone,abonelik
2,abone,aboneliği
3,absorbe,absorbe
4,absürt,absürt
...,...,...
4835,şövalye,şövalyenin
4836,şövalye,şövalyesi
4837,şırınga,şırınga
4838,şırınga,şırıngayla


In [112]:
df_all_word.search_word.nunique()

1558

In [113]:
df_all_word.word.nunique()

4711

In [114]:
df_shared_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency{file_ext}.xlsx")
#df_shared_all = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx")
df_shared_all

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,biraz,1269641.0,milyon dolar,6377,milyon dolardan fazla,279.0,servis modülü reaksiyon kontrol,20.0,radyo televizyon telefon bar video,10.0,sosisli sandviç,245.0
1,fazla,692043.0,biraz fazla,5111,standart kablolu orkestrası,191.0,fazla kaliteli normal banka,20.0,servis modülü reaksiyon kontrol sistemi,10.0,telefon numarası,232.0
2,kontrol,272328.0,telefon numarası,1473,kongre onur madalyası,113.0,projenin rutin kopya mikrofilm,17.0,modülü reaksiyon kontrol sistemi valfleri,10.0,standart prosedür,192.0
3,doktor,266714.0,biraz kahve,1354,milyon amerikan doları,84.0,frenler fren pedalları kablolar,14.0,NaN,NaN,genel alarm,139.0
4,polis,247969.0,milyon dolarlık,1102,milyar dolardan fazla,50.0,alfa alfa bravo eko,13.0,NaN,NaN,biraz fazla,138.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6765,NaN,NaN,sörf plajı,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6766,NaN,NaN,bebek gazla,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6767,NaN,NaN,sandviçleri paket,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6768,NaN,NaN,bebek fotoğrafı,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
df_word_order_twogram = word_in_wordgroup(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_wordgroup(df_shared_all, "word", "threegram") 
df_word_order_fourgram = word_in_wordgroup(df_shared_all, "word", "fourgram") 
df_word_order_fivegram = word_in_wordgroup(df_shared_all, "word", "fivegram")
df_word_order_sentence = word_in_wordgroup(df_shared_all, "word", "sentence") 

In [116]:
df_word_order_twogram = pd.merge(df_word_order_twogram,df_all_word, how="inner", on="word")
df_word_order_threegram = pd.merge(df_word_order_threegram,df_all_word, how="inner", on="word")
df_word_order_fourgram = pd.merge(df_word_order_fourgram,df_all_word, how="inner", on="word")
df_word_order_fivegram = pd.merge(df_word_order_fivegram,df_all_word, how="inner", on="word")
df_word_order_sentence = pd.merge(df_word_order_sentence,df_all_word, how="inner", on="word")

In [117]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["search_word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["search_word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["search_word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["search_word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["search_word"])["sentence"].apply(", ".join).reset_index()

In [118]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence]

In [119]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['search_word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all.rename(columns={"search_word":"word"}, inplace=True)
df_word_order_join_all

,word,twogram,threegram,fourgram,fivegram,sentence
0,abone,"gazeteye abone, polis abone, gazete aboneliği",NaN,NaN,NaN,"abone listesi, abone numarası"
1,acente,acente kopyası,NaN,NaN,NaN,NaN
2,adaptasyon,atmosfere adaptasyon,NaN,NaN,NaN,NaN
3,adrenalin,"doz adrenalin, adrenalin deposu, adrenalin fiz...",NaN,NaN,NaN,NaN
4,adres,"fatura adresi, posta adresi, adresi kontrol, a...","adresi telefon numarası, adres telefon numaras...",NaN,NaN,kod adresi
...,...,...,...,...,...,...
1108,offshore,NaN,NaN,NaN,NaN,offshore operasyonlar
1109,pelikan,NaN,NaN,NaN,NaN,pelikan flamingo
1110,sübjektif,NaN,NaN,NaN,NaN,biraz sübjektif
1111,termometre,NaN,NaN,NaN,NaN,diferansiyel termometre


In [120]:
# Option For Word Frequency
if shared_word_frequency:
    df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="left", on="word")
    df_word_order_join_all.drop_duplicates(inplace=True)
    df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram","fourgram","fivegram","sentence"]]
    df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
    df_word_order_join_all.reset_index(drop=True, inplace=True)
else:
    pass

df_word_order_join_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,kontrol,272328.0,"numarayı kontrol, trafik kontrol, kontrol pane...","reaksiyon kontrol sistemi, modülü reaksiyon ko...","servis modülü reaksiyon kontrol, modülü reaksi...","servis modülü reaksiyon kontrol sistemi, modül...","bilet kontrol, rutin kontrol, manuel kontrol, ..."
1,doktor,266714.0,"doktor fane, doktor raporu, doktor avukat, dok...",ambulans avukat doktor,"olimpiyat şampiyonu doktor üniversite, şampiyo...",NaN,"doktor fane, doktor park, bravo doktor, alo do..."
2,polis,247969.0,"polis departmanı, fazla polis, polis raporu, a...","metro polis departmanı, polis robot polis, met...",polis istasyonunda alarm otomatik,NaN,"alo polis, polis departmanı, süper polis, poli..."
3,dolar,199343.0,"milyon dolar, milyar dolar, milyonlarca dolar,...","dolar biraz fazla, milyon dolar fazla, milyon ...",NaN,NaN,"milyonlarca dolar, milyon dolar, milyar dolar,..."
4,alo,152645.0,"alo polis, alo radyo, alo ajan, alo gene, alo ...",NaN,NaN,NaN,"alo polis, alo doktor, alo operatör, alo radyo..."
...,...,...,...,...,...,...,...
1108,deodoran,11.0,marka deodorant,NaN,NaN,NaN,NaN
1109,aks,NaN,"biraz aksi, komisyon aksi, aksine ambulans, ak...",NaN,standart mekanik aksam standart,NaN,"biraz aksi, amerikan aksanı, fransız aksanı"
1110,barmen,NaN,"barmenlik kursu, barmenlik sertifikası, barmen...",NaN,NaN,NaN,NaN
1111,ekolayzer,NaN,ekolayzerin modern,NaN,NaN,NaN,NaN


In [121]:
df_word_order_join_all.word.nunique()

1113

In [122]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency{file_ext}3.xlsx", index=False)

#### Copy Move And Delete

In [123]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*{file_ext}3.xlsx")
output_file2

['Turkish_Italian_Shared_Join_Result_Without_Frequency43.xlsx']

In [124]:
for l in output_file2:
    source = l # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [125]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass

### Prefix Suffix Shared File Word Result Select

In [126]:
import os
import multiprocessing
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil

In [127]:
## language pair (same previous part parameter)
#lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "French"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# word sample
word_sample_num = 20

print(f"{file_ext}")

4


In [128]:
def lower_strip_func(x):
    try:
        var_low = x.lower()
        var_out = var_low.strip()
    except:
        var_out = x
    return var_out 

In [129]:
df_shared_process_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency{file_ext}3.xlsx")
df_shared_process_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,kontrol,272328.0,"numarayı kontrol, trafik kontrol, kontrol pane...","reaksiyon kontrol sistemi, modülü reaksiyon ko...","servis modülü reaksiyon kontrol, modülü reaksi...","servis modülü reaksiyon kontrol sistemi, modül...","bilet kontrol, rutin kontrol, manuel kontrol, ..."
1,doktor,266714.0,"doktor fane, doktor raporu, doktor avukat, dok...",ambulans avukat doktor,"olimpiyat şampiyonu doktor üniversite, şampiyo...",NaN,"doktor fane, doktor park, bravo doktor, alo do..."
2,polis,247969.0,"polis departmanı, fazla polis, polis raporu, a...","metro polis departmanı, polis robot polis, met...",polis istasyonunda alarm otomatik,NaN,"alo polis, polis departmanı, süper polis, poli..."
3,dolar,199343.0,"milyon dolar, milyar dolar, milyonlarca dolar,...","dolar biraz fazla, milyon dolar fazla, milyon ...",NaN,NaN,"milyonlarca dolar, milyon dolar, milyar dolar,..."
4,alo,152645.0,"alo polis, alo radyo, alo ajan, alo gene, alo ...",NaN,NaN,NaN,"alo polis, alo doktor, alo operatör, alo radyo..."
...,...,...,...,...,...,...,...
1108,deodoran,11.0,marka deodorant,NaN,NaN,NaN,NaN
1109,aks,NaN,"biraz aksi, komisyon aksi, aksine ambulans, ak...",NaN,standart mekanik aksam standart,NaN,"biraz aksi, amerikan aksanı, fransız aksanı"
1110,barmen,NaN,"barmenlik kursu, barmenlik sertifikası, barmen...",NaN,NaN,NaN,NaN
1111,ekolayzer,NaN,ekolayzerin modern,NaN,NaN,NaN,NaN


In [130]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all = df_word_all.loc[:,["word","frequency"]]
df_word_all["word"] = df_word_all["word"].apply(lambda x: lower_strip_func(x))
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988212,karneleme,5
988213,karnaya,5
988214,dörtlümüzün,5
988215,karnavalınız,5


In [131]:
df_twogram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")
df_twogram_all = df_twogram_all.loc[:,["twogram","frequency"]]
df_twogram_all["twogram"] = df_twogram_all["twogram"].apply(lambda x: lower_strip_func(x))
df_twogram_all

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
4457029,fikret cibran,3
4457030,romalı fikret,3
4457031,fikret ciooney,3
4457032,fikret cisco,3


In [132]:
df_threegram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")
df_threegram_all = df_threegram_all.loc[:,["threegram","frequency"]]
df_threegram_all["threegram"] = df_threegram_all["threegram"].apply(lambda x: lower_strip_func(x))
df_threegram_all

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
3009751,haydi büyük fikret,5
3009752,fikret caesardan beri,5
3009753,fikret cage kazandı,5
3009754,haydi bakayım fikret,5


In [133]:
df_fourgram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fourgram_Merge.csv")
df_fourgram_all = df_fourgram_all.loc[:,["fourgram","frequency"]]
df_fourgram_all["fourgram"] = df_fourgram_all["fourgram"].apply(lambda x: lower_strip_func(x))
df_fourgram_all

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
3052209,fikret miloya merhaba de,5
3052210,fikret millsin oğlu mu,5
3052211,fikret millet iyi iş,5
3052212,fikret millet bize bakıyor,5


In [134]:
df_fivegram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fivegram_Merge.csv")
df_fivegram_all = df_fivegram_all.loc[:,["fivegram","frequency"]]
df_fivegram_all["fivegram"] = df_fivegram_all["fivegram"].apply(lambda x: lower_strip_func(x))
df_fivegram_all

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
1096831,peder fikret hep şöyle söylerdi,4
1096832,peder fikret intihar etmeye çalışıyor,4
1096833,fikret dolson 12 gün yaşadı,4
1096834,ama fikret diye biri yoktu,4


In [135]:
df_sentence_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/Sentence/Merge/Sentence_Merge.csv")
df_sentence_all = df_sentence_all.loc[:,["sentence","frequency"]]
df_sentence_all["sentence"] = df_sentence_all["sentence"].apply(lambda x: lower_strip_func(x))
df_sentence_all

,sentence,frequency
0,evet,1948596
1,fikret,1533918
2,hayır,1250401
3,tamam,882921
4,ne,753105
...,...,...
2913965,hayır ben bir pikap kamyon alacağım,6
2913966,eminim uydurmuştur,6
2913967,hemen bir kement ile onu yakalar ve aşağı çekerim,6
2913968,tabii gerçek bir profesyonel o,6


In [136]:
for i in range(len(df_shared_process_all)):
    # column result
    try:
        # column result
        df_two_var = pd.DataFrame(df_shared_process_all.loc[i,"twogram"].split(", "), columns=["twogram"])
        # merge with all
        df_two_var_merge = pd.merge(df_two_var, df_twogram_all, how="left", on="twogram")
        df_two_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_two_var_merge_select = df_two_var_merge.head(word_sample_num)
        # to list value
        df_two_var_list = df_two_var_merge_select["twogram"].to_list()
        # list join
        df_two_var_list_join = ", ".join(df_two_var_list)
        # change value
        df_shared_process_all.loc[i,"twogram"] = df_two_var_list_join
    except:
        pass
    try:
        # column result
        df_three_var = pd.DataFrame(df_shared_process_all.loc[i,"threegram"].split(", "), columns=["threegram"])
        # merge with all
        df_three_var_merge = pd.merge(df_three_var, df_threegram_all, how="left", on="threegram")
        df_three_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_three_var_merge_select = df_three_var_merge.head(word_sample_num)
        # to list value
        df_three_var_list = df_three_var_merge_select["threegram"].to_list()
        # list join
        df_three_var_list_join = ", ".join(df_three_var_list)
        # change value
        df_shared_process_all.loc[i,"threegram"] = df_three_var_list_join
    except:
        pass
    try:
        # column result
        df_four_var = pd.DataFrame(df_shared_process_all.loc[i,"fourgram"].split(", "), columns=["fourgram"])
        # merge with all
        df_four_var_merge = pd.merge(df_four_var, df_fourgram_all, how="left", on="fourgram")
        df_four_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_four_var_merge_select = df_four_var_merge.head(word_sample_num)
        # to list value
        df_four_var_list = df_four_var_merge_select["fourgram"].to_list()
        # list join
        df_four_var_list_join = ", ".join(df_four_var_list)
        # change value
        df_shared_process_all.loc[i,"fourgram"] = df_four_var_list_join
    except:
        pass
    try:
        # column result
        df_five_var = pd.DataFrame(df_shared_process_all.loc[i,"fivegram"].split(", "), columns=["fivegram"])
        # merge with all
        df_five_var_merge = pd.merge(df_five_var, df_fivegram_all, how="left", on="fivegram")
        df_five_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_five_var_merge_select = df_five_var_merge.head(word_sample_num)
        # to list value
        df_five_var_list = df_five_var_merge_select["fivegram"].to_list()
        # list join
        df_five_var_list_join = ", ".join(df_five_var_list)
        # change value
        df_shared_process_all.loc[i,"fivegram"] = df_five_var_list_join
    except:
        pass
    try:
        # column result
        df_sentence_var = pd.DataFrame(df_shared_process_all.loc[i,"sentence"].split(", "), columns=["sentence"])
        # merge with all
        df_sentence_var_merge = pd.merge(df_sentence_var, df_sentence_all, how="left", on="sentence")
        df_sentence_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_sentence_var_merge_select = df_sentence_var_merge.head(word_sample_num)
        # to list value
        df_sentence_var_list = df_sentence_var_merge_select["sentence"].to_list()
        # list join
        df_sentence_var_list_join = ", ".join(df_sentence_var_list)
        # change value
        df_shared_process_all.loc[i,"sentence"] = df_sentence_var_list_join
    except:
        pass      

df_shared_process_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,kontrol,272328.0,"numarayı kontrol, biraz kontrolden, trafik kon...","reaksiyon kontrol sistemi, modülü reaksiyon ko...","servis modülü reaksiyon kontrol, modülü reaksi...","servis modülü reaksiyon kontrol sistemi, modül...","bilet kontrol, sistem kontrolü, rutin kontrol,..."
1,doktor,266714.0,"doktor fane, doktor raporu, doktorun numarası,...",ambulans avukat doktor,"olimpiyat şampiyonu doktor üniversite, şampiyo...",NaN,"doktor fane, doktor park, bravo doktor, alo do..."
2,polis,247969.0,"polis departmanı, trafik polisi, fazla polis, ...","düzine fransız polisi, metro polis departmanı,...",polis istasyonunda alarm otomatik,NaN,"alo polis, polis departmanı, metro polisi, tra..."
3,dolar,199343.0,"milyon dolar, milyon dolarlık, milyar dolar, d...","milyon dolardan fazla, milyon amerikan doları,...",NaN,NaN,"milyonlarca dolar, amerikan doları, milyon dol..."
4,alo,152645.0,"alo polis, alo radyo, alo ajan, alo gene, alo ...",NaN,NaN,NaN,"alo polis, alo doktor, alo operatör, alo radyo..."
...,...,...,...,...,...,...,...
1108,deodoran,11.0,marka deodorant,NaN,NaN,NaN,NaN
1109,aks,NaN,"amerikan aksanı, biraz aksi, aksine ambulans, ...",NaN,standart mekanik aksam standart,NaN,"biraz aksi, amerikan aksanı, fransız aksanı"
1110,barmen,NaN,"barmenlik kursu, otelin barmeni, barmenlik ser...",NaN,NaN,NaN,NaN
1111,ekolayzer,NaN,ekolayzerin modern,NaN,NaN,NaN,NaN


In [137]:
#df_shared_process_all.sort_values(by="frequency", ascending=False, inplace=True)

In [138]:
df_shared_process_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Select_Result_Without_Frequency{file_ext}4.xlsx", index=False)

#### Copy Move And Delete

In [139]:
output_file3 = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*_Select_*{file_ext}4.xlsx")
output_file3

['Turkish_Italian_Shared_Join_Select_Result_Without_Frequency44.xlsx']

In [140]:
for l in output_file3:
    source = l # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [141]:
for j in output_file3:
    try:
        os.remove(j)
    except:
        pass

### Word Count Analysis

In [ ]:
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

In [ ]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/Deploy Result Manuel").mkdir(parents=True, exist_ok=True)

In [ ]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

#### Hybrid Words

In [ ]:
df_hybrid = pd.read_excel("Turkish English manual selected 2 gram hybrids 3.xlsx", sheet_name="2 gram hybrid")
df_hybrid

In [ ]:
df_hybrid_count = word_count_result(df_hybrid, ["twogram_pair1","twogram_pair2","twogram_pair3","twogram_pair4"])
df_hybrid_count

In [ ]:
df_hybrid_count_merge = pd.merge(df_hybrid,df_hybrid_count,how="left",on="word")
df_hybrid_count_merge

In [ ]:
df_hybrid_count_merge2 = pd.merge(df_hybrid,df_hybrid_count,how="outer",on="word")
df_hybrid_count_merge2

In [ ]:
writer = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_Hybrid_Word_Count.xlsx", engine='xlsxwriter')

In [ ]:
df_hybrid_count_merge.to_excel(writer, sheet_name='28_Hybrid_Word_Count', index=False)
df_hybrid_count_merge2.to_excel(writer, sheet_name='All_Hybrid_Word_Count', index=False)

In [ ]:
writer.save()

#### Target Words

In [ ]:
df_target = pd.read_excel("Turkish English manual selected 2 gram hybrids 3.xlsx", sheet_name="2 gram target")
df_target

In [ ]:
df_target_count = word_count_result(df_target, ["twogram_1","twogram_2","twogram_3","twogram_4"])
df_target_count

In [ ]:
df_target_count_merge = pd.merge(df_target,df_target_count,how="left",on="word")
df_target_count_merge

In [ ]:
df_target_count_merge2 = pd.merge(df_target,df_target_count,how="outer",on="word")
df_target_count_merge2

In [ ]:
writer2 = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_Target_Word_Count.xlsx", engine='xlsxwriter')

In [ ]:
df_target_count_merge.to_excel(writer2, sheet_name='28_Target_Word_Count', index=False)
df_target_count_merge2.to_excel(writer2, sheet_name='All_Target_Word_Count', index=False)

In [ ]:
writer2.save()

#### All Target Hybrid Word Count 

In [ ]:
df_target_count

In [ ]:
df_hybrid_count

In [ ]:
df_all_word = pd.concat([df_target_count, df_hybrid_count], axis=0)
df_all_word

In [ ]:
df_all_word.groupby("word")[["word_count"]].sum().reset_index(inplace=True)
df_all_word.sort_values(by="word_count", ascending=False, inplace=True)
df_all_word

In [ ]:
df_all_word.to_excel(f"{lang_folder}_{lang_pair}_Target_Hybrid_Word_Count.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file4 = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_*_Word_Count.xlsx")
output_file4

In [ ]:
for o in output_file4:
    source = o # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [ ]:
for p in output_file4:
    try:
        os.remove(p)
    except:
        pass

### Youtube Search

In [28]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [29]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# parameter
sheets = "2 gram target"  # 2 gram target, 2 gram hybrids
time_shift = 0.3
sample_num = 6

In [30]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q
3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q
3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q
3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q


In [31]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [32]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3036932,471.970,472.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q
3036933,472.470,482.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q
3036934,482.498,484.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q
3036935,484.178,488.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q


In [33]:
df_word_group_select = pd.read_excel("Turkish English Manual Selected 2 Gram Hybrids.xlsx", sheet_name= "2 gram target")
df_word_group_select

,word,twogram_1,twogram_2,twogram_3,twogram_4
0,bir,bir şey,bir daha,bir de,bir çok
1,bu,bu kadar,bu ne,bu çok,bu değil
2,ne,ne kadar,ne için,bana ne,ne gibi
3,ve,ve de,ve evet,ve ne,ve sen
4,için,ne için,benim için,senin için,onun için
5,mi,değil mi,ben mi,sen mi,evet mi
6,o,o kadar,o da,o değil,evet o
7,ben,ben de,ben değil,hayır ben,ben mi
8,de,sen de,bir de,beni de,beni de
9,çok,bu çok,daha çok,çok var,çok şey


In [34]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[1:]:
        string = df_word_group_select.loc[i,j]
        search_list.append(string.strip())
search_list

['bir şey',
 'bir daha',
 'bir de',
 'bir çok',
 'bu kadar',
 'bu ne',
 'bu çok',
 'bu değil',
 'ne kadar',
 'ne için',
 'bana ne',
 'ne gibi',
 've de',
 've evet',
 've ne',
 've sen',
 'ne için',
 'benim için',
 'senin için',
 'onun için',
 'değil mi',
 'ben mi',
 'sen mi',
 'evet mi',
 'o kadar',
 'o da',
 'o değil',
 'evet o',
 'ben de',
 'ben değil',
 'hayır ben',
 'ben mi',
 'sen de',
 'bir de',
 'beni de',
 'beni de',
 'bu çok',
 'daha çok',
 'çok var',
 'çok şey',
 'ama bu',
 'evet ama',
 'hayır ama',
 'var ama',
 'var mı',
 'ne var',
 'daha var',
 'evet var',
 'evet ama',
 'evet bu',
 'evet ben',
 'evet o',
 'var mı',
 'hayır mı',
 'bana mı',
 'daha mı',
 'değil mi',
 'bu değil',
 'hayır değil',
 'o değil',
 'bu da',
 'o da',
 'daha da',
 'bana da',
 'bir şey',
 'bu şey',
 'o şey',
 'şey gibi',
 'hayır ben',
 'hayır mı',
 'hayır değil',
 'hayır yok',
 'bir daha',
 'daha çok',
 'daha var',
 'daha değil',
 'sen de',
 'sen mi',
 'hayır sen',
 'sen değil',
 'ne kadar',
 'bu kadar

In [35]:
def word_group_youtube(df,search_list,target_column, sample_num): 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]
        df_result.sort_values("sentence",key=lambda x:x.str.len(), inplace=True)
        df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    return df_search_result

In [12]:
#df_search_result = pd.DataFrame()
#for i in range(len(df_english_select)):
#    for j in df_english_select.columns[1:]:
#        string = df_english_select.loc[i,j]
#        df_result = df_youtube_sent[df_youtube_sent.sentence.str.contains(fr"(?:\s|^){string}(?:\s|$)", na=True)]
#        df_result.sort_values("sentence",key=lambda x:x.str.len(), inplace=True)
#        df_select = df_result.head(6)
#        df_select.insert(0,"search_string",string)
#        df_search_result = pd.concat([df_search_result,df_select], axis=0)
#df_search_result

In [36]:
df_search_result = word_group_youtube(df_youtube_sentence,search_list,"sentence",6)
df_search_result.reset_index(inplace=True, drop=True)
df_search_result

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,search_string,start_time,end_time,sentence,video_id
1413678,bir şey,5604.850,5605.350,bir şey,eHtgG748qhI
1989437,bir şey,3672.346,3672.991,bir şey,kCf15d2KbBo
1871310,bir şey,5681.980,5682.820,bir şey,8c6pH98yxyA
1986331,bir şey,851.790,852.790,bir şey,e2UAg5xF3Ks
2733360,bir şey,906.660,910.650,bir şey,qJd2PBgNCwU
...,...,...,...,...,...
1787629,yok gibi,4935.085,4936.095,yok gibi,cFBv6_tAXIo
2353191,yok gibi,3.919,5.101,ama yok gibi,2DzH7cHQPsE
2353305,yok gibi,375.819,376.921,ama yok gibi,2DzH7cHQPsE
2234154,yok gibi,988.533,989.515,evde yok gibi,dNbOwo0XPwE


In [20]:
df_search_result.columns

Index(['search_string', 'start_time', 'end_time', 'sentence', 'video_id'], dtype='object')

In [68]:
df_search_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 671 entries, 1413678 to 377252
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   search_string  671 non-null    object 
 1   start_time     671 non-null    float64
 2   end_time       671 non-null    float64
 3   sentence       671 non-null    object 
 4   video_id       671 non-null    object 
dtypes: float64(2), object(3)
memory usage: 47.6+ KB


In [44]:
word_time_loc_list = []
def word_group_time_loc(df):
    for i in range(len(df)):
        word = df.loc[i,"search_string"]
        start_time = df.loc[i,"{start_time"]
        end_time = df.loc[i,"end_time"]
        sentence = df.loc[i,"sentence"]
        video_id = df.loc[i,"video_id"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=["search_string","start_time","end_time","sentence","video_id"])

    return df_word_time_loc

In [45]:
word_group_time_loc(df_search_result)

KeyError: 0

In [46]:
sentence = "ama yok gibi"

In [61]:
a = re.finditer(fr"(?:\s|^)yok(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE)
a

In [65]:
b = []
for i in re.finditer(fr"(?:\s|^)yok gibi(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
    b.append(i)

In [66]:
b[0].start()

3

In [67]:
b[0].end()

12

In [ ]:
word_time_loc_list = []
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_id):
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_id}"])

    return df_word_time_loc

In [69]:
search_string = "search_string"
start_time = "start_time"
end_time = "end_time"
sentence = "sentence"
video_id = "video_id"

In [72]:
df_search_result

,search_string,start_time,end_time,sentence,video_id
1413678,bir şey,5604.850,5605.350,bir şey,eHtgG748qhI
1989437,bir şey,3672.346,3672.991,bir şey,kCf15d2KbBo
1871310,bir şey,5681.980,5682.820,bir şey,8c6pH98yxyA
1986331,bir şey,851.790,852.790,bir şey,e2UAg5xF3Ks
2733360,bir şey,906.660,910.650,bir şey,qJd2PBgNCwU
...,...,...,...,...,...
1787629,yok gibi,4935.085,4936.095,yok gibi,cFBv6_tAXIo
2353191,yok gibi,3.919,5.101,ama yok gibi,2DzH7cHQPsE
2353305,yok gibi,375.819,376.921,ama yok gibi,2DzH7cHQPsE
2234154,yok gibi,988.533,989.515,evde yok gibi,dNbOwo0XPwE


In [74]:
df_search_result.columns

Index(['search_string', 'start_time', 'end_time', 'sentence', 'video_id'], dtype='object')

In [75]:
df_search_result.loc[0,["start_time"]]

KeyError: 0

In [43]:
word_group_time_loc(df_search_result, search_string, start_time, end_time, sentence, video_id)

KeyError: 0

In [40]:

df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

KeyError: 0

In [ ]:
#time_shift = 0.3
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

In [ ]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 